In [1]:
import warnings
from dataclasses import replace
import numpy as np

warnings.filterwarnings('ignore')

#from helpers.functions import *
from helpers.variables import *
from helpers.helpers import *

from helpers.ipw import *
from helpers.propensity import get_propensity_scores

In [ ]:
# ================================================================
# Hyperparameter grids
# ================================================================

ALPHA_GRID = np.array([1e-5, 0.01, 0.025, 0.05, 0.07])
QUANTILE_GRID = np.array([0.005, 0.01, 0.015, 0.025, 0.05])
CLIPPING_GRID = [None, (1, 99), (2.5, 97.5)]

EXPOSURES = [ 
    "night_fat_g_target_day"
] # Should contain "_target_day" in their naming

# ================================================================
# Grid search loop
# ================================================================

for exposure in EXPOSURES:
    for alpha in ALPHA_GRID:

        cfg = replace(
            BASE_CONFIG,
            alpha=float(alpha),
        )

        # ---- propensity score estimation ----
        df, kwargs, X, shap_values = get_propensity_scores(
            exposure=exposure,
            config=cfg.__dict__,
            variables=variable_config,
            file=DATAFRAME_PATH,
            method="median",
            cutoff_values=None,
        )
        
        for q in QUANTILE_GRID:
            for clip in CLIPPING_GRID:
                # Create a new config instance instead of mutating dicts
                cfg = replace(
                    cfg,
                    q=float(q),
                    clip=clip
                )
                
                # ---- full causal experiment ----
                summary = run_experiment(
                    config=cfg.__dict__,
                    variable_config=variable_config,
                    df=df,
                    kwargs=kwargs,
                    X=X,
                    shap_values=shap_values,
                )

1e-05
Accuracy Score=0.627
[exp_id=1] exposure=night_fat_g | all_bal_0.10=71.4% | struct_bal_0.05=33.3% (unbal=2) | neg_controls_sig=0
[exp_id=2] exposure=night_fat_g | all_bal_0.10=75.0% | struct_bal_0.05=33.3% (unbal=2) | neg_controls_sig=0
[exp_id=3] exposure=night_fat_g | all_bal_0.10=85.7% | struct_bal_0.05=33.3% (unbal=2) | neg_controls_sig=0
[exp_id=4] exposure=night_fat_g | all_bal_0.10=85.7% | struct_bal_0.05=33.3% (unbal=2) | neg_controls_sig=0
[exp_id=5] exposure=night_fat_g | all_bal_0.10=89.3% | struct_bal_0.05=33.3% (unbal=2) | neg_controls_sig=0
[exp_id=6] exposure=night_fat_g | all_bal_0.10=92.9% | struct_bal_0.05=66.7% (unbal=1) | neg_controls_sig=0


In [8]:
df['treated'] = [
    'Early dinner' if i >= 2.8 else 'Late dinner'
    for i in df['hours_to_sleep_target_day']
]

In [13]:
columns_print = [
    "night_calories_pct_target_day",
    "night_calories_target_day",
    
    #"night_protein_pct_energy_target_day",
    #"night_carbs_pct_energy_target_day",
    #"night_fat_pct_energy_target_day",
    
    "last_meal_calories_target_day",
    "last_meal_calories_pct_target_day",

    "last_meal_protein_g_target_day",
    #"last_meal_protein_pct_target_day",

    "last_meal_carbs_g_target_day",
    #"last_meal_carbs_pct_target_day",

    "last_meal_fat_g_target_day",
    #"last_meal_fat_pct_target_day",
    
    #"dinner_carbs_percentage_target_day",
    #"fiber_g_per_1000_kcal_target_day",
    
    #"night_fat_g_target_day",
    
    "night_calories_pct_target_day",
    "night_calories_target_day",
    "night_prot_to_carb_target_day",
    
    "night_glp_target_day",
    
    "night_fcli_target_day",
    "night_cqi_lite_target_day",
    
    "night_sugars_g_target_day",
    "night_carbs_g_target_day",
    "night_protein_g_target_day",
    "night_fat_g_target_day",
]

In [14]:
df.groupby('treated').mean()[columns_print]

,night_calories_pct_target_day,night_calories_target_day,last_meal_calories_target_day,last_meal_calories_pct_target_day,last_meal_protein_g_target_day,last_meal_carbs_g_target_day,last_meal_fat_g_target_day,night_calories_pct_target_day,night_calories_target_day,night_prot_to_carb_target_day,night_glp_target_day,night_fcli_target_day,night_cqi_lite_target_day,night_sugars_g_target_day,night_carbs_g_target_day,night_protein_g_target_day,night_fat_g_target_day
treated,,,,,,,,,,,,,,,,,
Early dinner,29.515222,532.181498,566.181489,32.916027,25.178689,57.942920,25.184071,29.515222,532.181498,9854.491280,49.077135,0.227948,-127.426618,18.556956,55.163345,23.068945,23.625741
Late dinner,40.411516,762.280645,373.253310,20.323237,14.725701,40.530087,16.278022,40.411516,762.280645,3277.976523,74.221683,0.228060,-0.280127,30.898693,83.274890,31.416633,32.825269
